In [ ]:
# importamos wget y las librerías que nos faltan
from wget import download
from os import path
import pandas as pd
import numpy as np
#https://github.com/agustinbx/DataSet-Trabajo/winequality_BR.csv



# descargamos la base de datos
if not path.exists("winequality_BR.csv"):
  download("https://github.com/agustinbx/DataSet-Trabajo/raw/main/winequality_BR.csv")
else:
  print("No vamos a bajar el archivo de itunes porque ya existe!")

In [ ]:
raw_ds = pd.read_csv("winequality_BR.csv", delimiter=';')
winesDf = raw_ds.copy()
winesDf

Elimino las columnas que son nulas y las id ya que son innecesarias y complican el analisis

In [ ]:
winesDf= winesDf.dropna(axis=1,how="all")
winesDf

Inicio el analisis de los datos, primero necesito saber con que tipos de datos estoy trabajando y si estan en buen estado. ademas necesito saber si tengo valores faltantantes por lo que tambien corroboro que valores nulos tengo.

In [ ]:
winesDf.isna().sum() #con esta linea de codigo verifico cuantos valores nulos tenemos en el csv y los sumo

Este resultado indica que no hay valores NaN en el csv. Por lo que continuo con el analisis de columnas.

In [ ]:
winesDf.info()

type: tipo de uva con la que se elabora el vino. 

fixed acidity: cantidad de ácidos no volátiles presentes en el vino, medida en gramos por litro.

volatile acidity: cantidad de ácidos volátiles presentes en el vino, medida en gramos por litro. 

citric acid: contenido de ácido cítrico en el vino, medido en gramos por litro.

residual sugar: cantidad de azúcar que queda en el vino después de la fermentación, medida en gramos por litro. 

chlorides: concentración de cloruros (sales) en el vino, medida en gramos por litro.

free sulfur dioxide: cantidad de dióxido de azufre que no está ligado químicamente en el vino, medida en miligramos por litro. 

total sulfur dioxide: suma del dióxido de azufre libre y el combinado en el vino, medida en miligramos por litro.

density: medida de la masa por unidad de volumen del vino, utilizada para estimar la concentración de sólidos disueltos, medida en gramos por centímetro cúbico.

pH: medida de la acidez o alcalinidad del vino. 

sulphates: concentración de sales de sulfato en el vino, medida en gramos por litro.

alcohol: contenido alcohólico del vino, medido en porcentaje de volumen (% vol).

quality: puntuación del vino, con una escala que va de 0 a 10.


Hay una inconsistencia y una mejora posible para los datos
    
    -En cuanto a la columna type, esta es representada por pandas como object debido a que contiene strings, pero al ser estas strings siempre las mismas pandas permite tranformar el tipo de dato a category el cual es mas eficiente en estos casos en terminos de rendimiento.

    -En lo que a la columna alcohol refiere esta esta representada por el tipo object siendo que esta deberia ser de float64, esto posiblemente se deba a que alcohol contenga valores que pandas no puede intepretar como numeros. Por lo cual se debe hallar estos valores y decidir que hacer con ellos.

    Garnacha;11.6;0.475;0.4;1.4;0.091;6;28;0.99704;3.07;0.65;100.333.333.333.333;6;;;
    Garnacha;11.6;0.475;0.4;1.4;0.091;6;28;0.99704;3.07;0.65;100.333.333.333.333;6;;;


In [ ]:
#convierto los datos erroneos en NaN
winesDf["alcohol"] = pd.to_numeric(winesDf["alcohol"],errors='coerce')

winesDf.info()

In [ ]:
#calculo el promedio de % de volumen alcoholico entre los vinos de los cuales si tengo los datos
prom = winesDf['alcohol'].mean()
winesDf['alcohol'].fillna(prom,inplace= True)
winesDf.info()

In [ ]:
winesDf.loc[winesDf['density'] > 2, 'density'] = np.nan
mean_density = winesDf['density'].mean()
winesDf['density'].fillna(mean_density, inplace=True)
winesDf.info()

In [ ]:
import pandas_profiling as pp

#generamos el reporte
report = pp.ProfileReport(winesDf)

# lo mostramos interactivo en nuestra notebook
#report.to_notebook_iframe()

#descarga el archivo html (lo abrimos desde google)
#report.to_file(output_file="Profile_report_winequality.html")



La correlación es un concepto estadístico que indica la dirección y la fuerza de la relación entre dos variables. Cuando dos variables están correlacionadas, significa que los cambios en una de ellas están asociados con cambios en la otra. Esta asociación puede ser positiva (cuando una variable aumenta, la otra también lo hace) o negativa (cuando una variable aumenta, la otra disminuye).

El coeficiente de correlación de Pearson es la medida más utilizada para cuantificar esta relación. Se representa con la letra  r  y toma valores entre -1 y 1. Un valor de  r  cercano a 1 indica una fuerte correlación positiva, mientras que un valor cercano a -1 indica una fuerte correlación negativa. Un valor cercano a 0 sugiere que no hay una correlación lineal evidente entre las variables.

Correlación no implica causalidad. Dos variables pueden estar correlacionadas sin que una cause necesariamente la otra.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Generamos la matriz de correlacion, teniendo en cuenta unicamente valores numericos
numeric_data = winesDf.select_dtypes(include=[np.number])
#numeric_data = numeric_data.dropna()
correlation_matrix = numeric_data.corr()

#correlation_matrix


print(correlation_matrix['quality'].sort_values(ascending=False))


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline
#imprimimos la matriz como un heatmap
plt.figure(figsize=(16,12))
sns.heatmap(correlation_matrix, vmin=-1.0, vmax=1.0, center=0.0, annot=True, cmap='coolwarm')
plt.title('Heatmap de la Matriz de Correlación', fontsize=18)
plt.show()

Una regla de esas que no tienen demostración pero que funciona bastante bien (el término en inglés para definirlas es "rules of thumb") dice que si la correlación da por encima de 0.7 (o por debajo de -0.7), la relación entre esas dos variables merece un poco de análisis extra.

In [ ]:
# tomamos el valor absoluto de las correlaciones, umbralamos las mayores a 0.7
correlation_matrix_umbralizada = correlation_matrix.abs() > 0.7 

# aprovechamos y sacamos la diagonal
np.fill_diagonal(correlation_matrix_umbralizada.values, 0)

# e imprimimos la matriz como un heatmap
plt.figure(figsize=(16,12))
sns.heatmap(correlation_matrix_umbralizada, vmin=0.0, vmax=1.0, center=0.0, annot=True, cmap= 'coolwarm')
plt.show()

In [ ]:
# iteramos por cada una de las columnas
for i, col in enumerate(correlation_matrix_umbralizada.columns):
  # y por cada una de las filas (para eso usamos el índice)
  for j in range(i,len(correlation_matrix_umbralizada.index)):
    # extraemos el nombre de la fila en la que estamos
    row = correlation_matrix_umbralizada.index[j]
    # si en esa posición tenemos un True en la matriz umbralizada
    if correlation_matrix_umbralizada.loc[row, col]:
      # nos traemos el valor de esa correlación
      correlation = correlation_matrix.loc[row, col]
      # representamos en un scatter plot cuadrado los valores de ambas variables
      plt.figure(figsize=(6, 6))
      plt.scatter(winesDf[row], winesDf[col])
      # y lo mostramos bonito
      plt.title(f"Scatter plot entre {col} y {row} - r = {correlation:.4f}")
      plt.xlabel(row)
      plt.ylabel(col)
      plt.grid(True)
      plt.show()    

In [ ]:
print(winesDf.columns)

OUTLIER ??????

In [ ]:
plt.figure(figsize=(20,6))  # Opcional, ajustar el tamaño de la figura
sns.boxplot(x=winesDf['citric acid'])  # Crear el Box Plot

plt.xlim(0, 2)  # Escala del eje x

plt.title('Box Plot de Citric Acid')  # Añadir título
plt.show()  # Mostrar el gráfico

In [ ]:
# Crear el gráfico de dispersión
plt.figure(figsize=(8,6))  # Ajustar el tamaño del gráfico
sns.scatterplot(x='pH', y='quality', hue='type', data=winesDf, palette='Set1')  # Usar hue para diferenciar por type

# Añadir títulos y etiquetas
plt.title('Relación entre pH y quality separada por type')
plt.xlabel('pH')
plt.ylabel('quality')

plt.show()


In [ ]:
plt.figure(figsize=(8,6))
sns.scatterplot(x='pH', y='quality', hue='type', data=winesDf, palette='Set2')

plt.title('Relación entre pH y quality separada por type')
plt.xlabel('pH')
plt.ylabel('Quality')

plt.show()

RELACION DE DENSIDAD Y SULFATOS SEPARADO POR TIPO.

In [ ]:
tintos = winesDf[winesDf['type'] == 'Garnacha']
blancos = winesDf[winesDf['type'] == 'Riesling']

# Crear un FacetGrid
g = sns.FacetGrid(winesDf, col='type', height=6, aspect=1)
g.map(sns.scatterplot, 'sulphates', 'density')

# Añadir etiquetas y título
g.set_axis_labels('sulf', 'dens')
g.fig.suptitle('Relación entre pH y Quality separada por Type', y=1.05)

plt.show()



residual sugar: cantidad de azúcar que queda en el vino después de la fermentación, medida en gramos por litro.

Los vinos blancos tienen mayor cantidad de azúcar residual (residual sugar) que los vinos tintos.

Justificación: Los vinos blancos suelen ser más dulces que los tintos, lo que podría reflejarse en mayores niveles de azúcar residual. 

Explayar mas con los datos del grafico y el describe.

HIPOTESIS FALOPA, NOSE SI ES LA IDEA. CAPAZ SIRVE.



In [ ]:
tintos = winesDf[winesDf['type'] == 'Garnacha']
blancos = winesDf[winesDf['type'] == 'Riesling']


# Calcular estadísticas descriptivas de la acidez fija
print("Azucar Residual en vinos tintos (Garnacha):")
print(tintos['residual sugar'].describe())

print("\nAzucar Residual en vinos blancos (Riesling):")
print(blancos['residual sugar'].describe())


import seaborn as sns
import matplotlib.pyplot as plt

# Crear un boxplot para comparar la acidez fija en ambos tipos de vino
sns.boxplot(x='type', y='residual sugar', data=winesDf)
plt.title('Comparación de Residual Sugar entre Vinos Tintos (Garnacha) y Blancos (Riesling)')
plt.xlabel('Tipo de Vino')
plt.ylabel('Residual Sugar')
plt.show()

free sulfur dioxide: cantidad de dióxido de azufre que no está ligado químicamente en el vino, medida en miligramos por litro. 

El dióxido de azufre libre (free sulfur dioxide) es más alto en los vinos blancos que en los tintos.

Justificación: El dióxido de azufre es un conservante comúnmente más usado en los vinos blancos para evitar la oxidación.

OTRA HIPOTESIS FALOPA, CAPAZ SIRVE, ANALIZAR EL GRAFICO, DESCRIBE, BUSCAR INFO DE LA VARIABLE


In [ ]:
# Calcular estadísticas descriptivas de la acidez fija
print("Dioxido de azufre libre en vinos tintos (Garnacha):")
print(tintos['free sulfur dioxide'].describe())

print("\nDioxido de azufre libre en vinos blancos (Riesling):")
print(blancos['free sulfur dioxide'].describe())


import seaborn as sns
import matplotlib.pyplot as plt

# Crear un boxplot para comparar la acidez fija en ambos tipos de vino
sns.boxplot(x='type', y='free sulfur dioxide', data=winesDf)
plt.title('Comparación de Dioxido de azufre libre entre Vinos Tintos (Garnacha) y Blancos (Riesling)')
plt.xlabel('Tipo de Vino')
plt.ylabel('Free Sulfur Dioxide')
plt.show()

In [ ]:
# Crear el scatter plot
plt.figure(figsize=(10, 6))
sns.scatterplot(x='free sulfur dioxide', y='total sulfur dioxide', hue='type', data=winesDf, palette='Set2')

# Añadir título y etiquetas
plt.title('Relación entre Residual Sugar y Quality separada por Type')
plt.xlabel('density')
plt.ylabel('Quality')

# Mostrar el gráfico
plt.show()

In [ ]:
# Crear un FacetGrid
g = sns.FacetGrid(winesDf, col='type', height=6, aspect=1)
g.map(sns.scatterplot, 'free sulfur dioxide', 'total sulfur dioxide')

# Añadir etiquetas y título
g.set_axis_labels('Residual Sugar', 'Quality')
g.fig.suptitle('Relación entre Residual Sugar y Quality separada por Type', y=1.05)

plt.show()

In [ ]:
# Crear un FacetGrid
g = sns.FacetGrid(winesDf, col='type', height=6, aspect=1)
g.map(sns.scatterplot, 'citric acid', 'volatile acidity')

# Añadir etiquetas y título
g.set_axis_labels('Residual Sugar', 'Quality')
g.fig.suptitle('Relación entre Residual Sugar y Quality separada por Type', y=1.05)

plt.show()

HIPOTESIS

Los vinos que utilizan el tipo de uva Garnacha son mas acidos que los vinos de tipo de uva Riesling?

Analizamos las columnas Fixed Acidity y Volatile Acidity separados por tipo.

En primer lugar vemos graficamente mediante box-plots que ocurre para cada caso.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Crear un boxplot para comparar la acidez fija en ambos tipos de vino
sns.boxplot(x='type', y='fixed acidity', data=winesDf, notch=True)
plt.title('Comparación de fixed acidity entre Vinos Tintos (Garnacha) y Blancos (Riesling)')
plt.xlabel('Tipo de Vino')
plt.ylabel('fixed acidity')
plt.show()

# Crear un boxplot para comparar la acidez fija en ambos tipos de vino
sns.boxplot(x='type', y='volatile acidity', data=winesDf, notch=True)
plt.title('Comparación de volatile acidity entre Vinos Tintos (Garnacha) y Blancos (Riesling)')
plt.xlabel('Tipo de Vino')
plt.ylabel('volatile acidity')
plt.show()

# Crear un boxplot para comparar la acidez fija en ambos tipos de vino
sns.boxplot(x='type', y='citric acid', data=winesDf, notch=True)
plt.title('Comparación de volatile acidity entre Vinos Tintos (Garnacha) y Blancos (Riesling)')
plt.xlabel('Tipo de Vino')
plt.ylabel('citric acid')
plt.show()



Notamos una diferencia en los graficos.

Para validar la hipotesis, podemos usar test T, pero primero hay que validar Normalidad de los datos a traves de Shapiro Wilk

In [ ]:
from scipy.stats import shapiro

# Separamos los datos en dos grupos, desarrollados y en vías de desarrollo
garnacha = winesDf[winesDf['type'] == 'Garnacha']['fixed acidity']
riesling = winesDf[winesDf['type'] == 'Riesling']['fixed acidity']

# Test de Shapiro-Wilk para países desarrollados
stat, p = shapiro(garnacha)
print(f"Test de Shapiro-Wilk para vinos garnacha: Estadístico={stat:.3f}, p-valor={p:.3f}")

# Test de Shapiro-Wilk para países en vías de desarrollo
stat, p = shapiro(riesling)
print(f"Test de Shapiro-Wilk para vinos riesling: Estadístico={stat:.3f}, p-valor={p:.3f}")

Como obtenemos un p-valor menor a 0.05, estamos ante datos que no respetan una distribucion normal.

(tambien se puede probar por Kolmogorov-Smirnov. Este test es mas general y menos potente para detectar desviaciones sutiles de normalidad)



In [ ]:
from scipy.stats import kstest

# Test de Kolmogorov-Smirnov para países desarrollados
stat, p = kstest(garnacha, 'norm')
print(f"Test de Kolmogorov-Smirnov para vinos riesling: Estadístico={stat:.3f}, p-valor={p:.3f}")

# Test de Kolmogorov-Smirnov para países en vías de desarrollo
stat, p = kstest(riesling, 'norm')
print(f"Test de Kolmogorov-Smirnov para vinos riesling: Estadístico={stat:.3f}, p-valor={p:.3f}")

Analizamos el QQ plot. Si los puntos estan muy cerca de la linea, hay normalidad.



In [ ]:
import matplotlib.pyplot as plt
import scipy.stats as stats

# QQ plot para países desarrollados
stats.probplot(garnacha, dist="norm", plot=plt)
plt.title("QQ Plot para fixed acidity en vinos garnacha")
plt.show()

# QQ plot para países en vías de desarrollo
stats.probplot(riesling, dist="norm", plot=plt)
plt.title("QQ Plot para fixed acidity en vinos riesling")
plt.show()

Como ninguno de estos casos nos dieron lo que queremos, no podemos usar test T.

Probamos homocedasticidad (homogeneidad de varianzas) utilizando Levene. 

In [ ]:
stat, p = stats.levene(garnacha, riesling)
print(f"Test de Levene para fixed acidity: Estadístico={stat:.3f}, p-valor={p:.3f}")

Como nos dio un p-valor menor a 0.05 tenemos datos que no son homocedasticos.

Validamos con Kruskal Wallis (con alpha 0.05)

In [ ]:
# Test de Kruskal-Wallis para comparar GDP_per_capita entre países desarrollados y en vías de desarrollo
stat, p = stats.kruskal(garnacha, riesling)
print(f"Test de Kruskal-Wallis para fixed acidity: Estadístico={stat:.3f}, p-valor={p:.3f}")

# Interpretación de los resultados
alpha = 0.05  # Nivel de significancia
if p > alpha:
    print("No hay suficiente evidencia para rechazar la hipótesis nula.")
    print("No hay una diferencia significativa en el fixed acidity entre vinos garnacha y vinos riesling.")
else:
    print("Se rechaza la hipótesis nula.")
    print("Existe una diferencia significativa en el fixed acidity entre vinos garnacha y vinos riesling.")

como nos dio un p-valor muy chico existen diferencias estadisticamente significativas entre los valores que estamos comparando.


Hacemos lo mismo para la variable volatile acidity

Box-Plot

Shapiro Wilk

QQ plot

Levene 

Kruskal wallis


In [ ]:
# Separamos los datos en dos grupos, desarrollados y en vías de desarrollo
garnacha2 = winesDf[winesDf['type'] == 'Garnacha']['volatile acidity']
riesling2 = winesDf[winesDf['type'] == 'Riesling']['volatile acidity']

# Test de Shapiro-Wilk para países desarrollados
stat, p = shapiro(garnacha2)
print(f"Test de Shapiro-Wilk para vinos garnacha: Estadístico={stat:.3f}, p-valor={p:.3f}")

# Test de Shapiro-Wilk para países en vías de desarrollo
stat, p = shapiro(riesling2)
print(f"Test de Shapiro-Wilk para vinos riesling: Estadístico={stat:.3f}, p-valor={p:.3f}")

In [ ]:
# QQ plot para países desarrollados
stats.probplot(garnacha2, dist="norm", plot=plt)
plt.title("QQ Plot para volatile acidity en vinos garnacha")
plt.show()

# QQ plot para países en vías de desarrollo
stats.probplot(riesling2, dist="norm", plot=plt)
plt.title("QQ Plot para volatile acidity en vinos riesling")
plt.show()

In [ ]:
stat, p = stats.levene(garnacha2, riesling2)
print(f"Test de Levene para volatile acidity: Estadístico={stat:.3f}, p-valor={p:.3f}")


# Test de Kruskal-Wallis para comparar GDP_per_capita entre países desarrollados y en vías de desarrollo
stat, p = stats.kruskal(garnacha2, riesling2)
print(f"Test de Kruskal-Wallis para volatile acidity: Estadístico={stat:.3f}, p-valor={p:.3f}")

# Interpretación de los resultados
alpha = 0.05  # Nivel de significancia
if p > alpha:
    print("No hay suficiente evidencia para rechazar la hipótesis nula.")
    print("No hay una diferencia significativa en el volatile acidity entre vinos garnacha y vinos riesling.")
else:
    print("Se rechaza la hipótesis nula.")
    print("Existe una diferencia significativa en el volatile acidity entre vinos garnacha y vinos riesling.")


VAMOS A PROBAR ACIDEZ TANTO EN LO VINOS RIESLING COMO GARNACHA A TRAVES DE LA COLUMNA 'PH'

In [ ]:
# Crear un boxplot para comparar la acidez fija en ambos tipos de vino
sns.boxplot(x='type', y='pH', data=winesDf, notch=True)
plt.title('Comparación de volatile acidity entre Vinos Tintos (Garnacha) y Blancos (Riesling)')
plt.xlabel('Tipo de Vino')
plt.ylabel('pH')
plt.show()


In [ ]:
# Separamos los datos en dos grupos, Garnacha y Riesling
garnachaPH = winesDf[winesDf['type'] == 'Garnacha']['pH']
rieslingPH = winesDf[winesDf['type'] == 'Riesling']['pH']

# Test de Shapiro-Wilk para países desarrollados
stat, p = shapiro(garnachaPH)
print(f"Test de Shapiro-Wilk para vinos garnacha: Estadístico={stat:.3f}, p-valor={p:.3f}")

# Test de Shapiro-Wilk para países en vías de desarrollo
stat, p = shapiro(rieslingPH)
print(f"Test de Shapiro-Wilk para vinos riesling: Estadístico={stat:.3f}, p-valor={p:.3f}")

In [ ]:
# Test de Kolmogorov-Smirnov para vinos Garnacha
stat, p = kstest(garnachaPH, 'norm')
print(f"Test de Kolmogorov-Smirnov para vinos riesling: Estadístico={stat:.3f}, p-valor={p:.3f}")

# Test de Kolmogorov-Smirnov para vinos Riesling
stat, p = kstest(rieslingPH, 'norm')
print(f"Test de Kolmogorov-Smirnov para vinos riesling: Estadístico={stat:.3f}, p-valor={p:.3f}")

In [ ]:
# QQ plot para países desarrollados
stats.probplot(garnachaPH, dist="norm", plot=plt)
plt.title("QQ Plot para residual sugar en vinos garnacha")
plt.show()

# QQ plot para países en vías de desarrollo
stats.probplot(rieslingPH, dist="norm", plot=plt)
plt.title("QQ Plot para residual sugar en vinos riesling")
plt.show()

In [ ]:
stat, p = stats.levene(garnachaPH, rieslingPH)
print(f"Test de Levene para ph: Estadístico={stat:.3f}, p-valor={p:.3f}")

# Interpretación de los resultados
alpha = 0.05  # Nivel de significancia
if p > alpha:
    print("No hay suficiente evidencia para rechazar la hipótesis nula.")
    print("Las varianzas de ph son iguales entre garnacha y riesling.")
else:
    print("Se rechaza la hipótesis nula.")
    print("Las varianzas de ph son diferentes entre garnacha y riesling.")

In [ ]:
# Test t para comparar la longitud del sépalo entre Setosa y Virginica
t_statistic, p_value = stats.ttest_ind(garnachaPH, rieslingPH)
print(f"Test t para PH: Estadístico={t_statistic:.3f}, p-valor={p_value:.3f}")

# Interpretación de los resultados
alpha = 0.05  # Nivel de significancia
if p_value > alpha:
    print("No hay suficiente evidencia para rechazar la hipótesis nula.")
    print("No hay una diferencia significativa en el PH entre garnacha y riesling.")
else:
    print("Se rechaza la hipótesis nula.")
    print("Existe una diferencia significativa en el PH entre garnacha y riesling.")

In [ ]:
import scipy.stats as stats

# Test t para comparar la longitud del sépalo entre Setosa y Virginica
t_statistic, p_value = stats.ttest_ind(garnachaPH, rieslingPH, alternative='less')

print(f"Test t a una cola para PH: Estadístico={t_statistic:.3f}, p-valor={p_value:.3f}")

# Interpretación de los resultados
alpha = 0.05  # Nivel de significancia
if p_value > alpha:
    print("No hay suficiente evidencia para rechazar la hipótesis nula.")
    print("No hay una diferencia significativa en el PH entre garnacha y riesling.")
else:
    print("Se rechaza la hipótesis nula.")
    print("El ph de garnacha es significativamente menor que la de riesling.")

In [ ]:
# Test de Kruskal-Wallis para comparar GDP_per_capita entre países desarrollados y en vías de desarrollo
stat, p = stats.kruskal(garnachaPH, rieslingPH)
print(f"Test de Kruskal-Wallis para ph: Estadístico={stat:.3f}, p-valor={p:.3f}")

# Interpretación de los resultados
alpha = 0.05  # Nivel de significancia
if p > alpha:
    print("No hay suficiente evidencia para rechazar la hipótesis nula.")
    print("No hay una diferencia significativa en el ph entre vinos garnacha y vinos riesling.")
else:
    print("Se rechaza la hipótesis nula.")
    print("Existe una diferencia significativa en el ph entre vinos garnacha y vinos riesling.")

COMPARACION ENTRE TIPOS DE VINOS POR LA VARIABLE 'RESIDUAL SUGAR'

LOS VINOS RIESLING SON MAS DULCE QUE LOS GARNACHA (o los vinos riesling contienen mas residual sugar que los garnacha)

analizar mejor en que se basa que un vino sea mas dulce que otro (nose si solo se basa en residual sugar)

In [ ]:
# Crear un boxplot para comparar la residual sugar en ambos tipos de vino
sns.boxplot(x='type', y='residual sugar', data=winesDf, notch=True)
plt.title('Comparación de residual sugar entre Vinos Tintos (Garnacha) y Blancos (Riesling)')
plt.xlabel('Tipo de Vino')
plt.ylabel('residual sugar')
plt.show()

ANALIZAMOS NORMALIDAD 

PROBAMOS POR SHAPIRO - WILK, si el p-valor que obtenemos es menor a nuestro nivel de confianza  α  (normalmente  α=0.05 ), estamos entonces ante datos que no respetan una distribución normal:

In [ ]:
# Separamos los datos en dos grupos, Garnacha y Riesling
garnacha3 = winesDf[winesDf['type'] == 'Garnacha']['residual sugar']
riesling3 = winesDf[winesDf['type'] == 'Riesling']['residual sugar']

# Test de Shapiro-Wilk para países desarrollados
stat, p = shapiro(garnacha3)
print(f"Test de Shapiro-Wilk para vinos garnacha: Estadístico={stat:.3f}, p-valor={p:.3f}")

# Test de Shapiro-Wilk para países en vías de desarrollo
stat, p = shapiro(riesling3)
print(f"Test de Shapiro-Wilk para vinos riesling: Estadístico={stat:.3f}, p-valor={p:.3f}")

SEGUIMOS PROBANDO MEDIANTE KOLMOGOROV SMIRNOV



In [ ]:
# Test de Kolmogorov-Smirnov para vinos Garnacha
stat, p = kstest(garnacha3, 'norm')
print(f"Test de Kolmogorov-Smirnov para vinos riesling: Estadístico={stat:.3f}, p-valor={p:.3f}")

# Test de Kolmogorov-Smirnov para vinos Riesling
stat, p = kstest(riesling3, 'norm')
print(f"Test de Kolmogorov-Smirnov para vinos riesling: Estadístico={stat:.3f}, p-valor={p:.3f}")

QQ PLOT

se basa en qué tan cerca están los puntos de una línea recta: si están muy cerca, hay normalidad; si no, no la hay

In [ ]:
# QQ plot para países desarrollados
stats.probplot(garnacha3, dist="norm", plot=plt)
plt.title("QQ Plot para residual sugar en vinos garnacha")
plt.show()

# QQ plot para países en vías de desarrollo
stats.probplot(riesling3, dist="norm", plot=plt)
plt.title("QQ Plot para residual sugar en vinos riesling")
plt.show()

COMO NINGUN METODO NOS CONFIRMO QUE HAY NORMALIDAD NO PODEMOS USAR TEST T

PROBAMOS HOMOCEDASTICIDAD MEDIANTE LEVENE

Ojo que en Levene la hipótesis nula es que las varianzas son significativamente diferentes entre sí (hay heterocedasticidad), por lo que pedimos que el test nos de un p-valor mayor a 0.05:

In [ ]:
stat, p = stats.levene(garnacha3, riesling3)
print(f"Test de Levene para residual sugar: Estadístico={stat:.3f}, p-valor={p:.3f}")


Dado que nuestro conjunto no cumple con ninguno de los supuestos que se necesitan para realizar la comparación de estos grupos, vamos a tener que recurrir al test de Kruskal-Wallis. La hipótesis nula del test es que no existen diferencias significativas entre los dos grupos provistos. Por consiguiente, si el p-valor del test nos da por debajo del umbral de significancia (acá abajo les puse una variable alpha en 0.05 para controlar eso), efectivamente hay una diferencia significativa entre ambos.

In [ ]:
# Test de Kruskal-Wallis para comparar GDP_per_capita entre países desarrollados y en vías de desarrollo
stat, p = stats.kruskal(garnacha3, riesling3)
print(f"Test de Kruskal-Wallis para residual sugar: Estadístico={stat:.3f}, p-valor={p:.3f}")

# Interpretación de los resultados
alpha = 0.05  # Nivel de significancia
if p > alpha:
    print("No hay suficiente evidencia para rechazar la hipótesis nula.")
    print("No hay una diferencia significativa en el residual sugar entre vinos garnacha y vinos riesling.")
else:
    print("Se rechaza la hipótesis nula.")
    print("Existe una diferencia significativa en el residual sugar entre vinos garnacha y vinos riesling.")

In [ ]:
import pandas as pd

# Supongamos que tu DataFrame original se llama winesDf
# Crear una copia del DataFrame
winesDf_copy = winesDf.copy()

# Añadir la columna 'quality_group' en la copia
winesDf_copy['quality_group'] = pd.cut(winesDf_copy['quality'], 
                                       bins=[0, 5, 7, 10], 
                                       labels=['Baja', 'Media', 'Alta'])

# de 0 a 4 CALIDAD BAJA
# de 5 a 6 CALIDAD MEDIA
# de 7 a 10 CALIDAD ALTA

In [ ]:
winesDf_copy['quality_group'].value_counts()

In [ ]:
# Crear un boxplot para comparar la acidez fija en ambos tipos de vino
sns.boxplot(x='quality_group', y='alcohol', data=winesDf_copy, notch=True)
plt.title('Comparación de alcohol entre calidad de vino')
plt.xlabel('Calidad')
plt.ylabel('alcohol')
plt.show()

In [ ]:
# Separamos los datos en dos grupos, desarrollados y en vías de desarrollo
vino_q_baja = winesDf_copy[winesDf_copy['quality_group'] == 'Baja']['alcohol']
vino_q_media = winesDf_copy[winesDf_copy['quality_group'] == 'Media']['alcohol']
vino_q_alta = winesDf_copy[winesDf_copy['quality_group'] == 'Alta']['alcohol']

# Test de Shapiro-Wilk para países desarrollados
stat, p = shapiro(vino_q_baja)
print(f"Test de Shapiro-Wilk para vinos Q baja: Estadístico={stat:.3f}, p-valor={p:.3f}")

# Test de Shapiro-Wilk para países en vías de desarrollo
stat, p = shapiro(vino_q_media)
print(f"Test de Shapiro-Wilk para vinos Q media: Estadístico={stat:.3f}, p-valor={p:.3f}")

# Test de Shapiro-Wilk para países en vías de desarrollo
stat, p = shapiro(vino_q_alta)
print(f"Test de Shapiro-Wilk para vinos Q alta: Estadístico={stat:.3f}, p-valor={p:.3f}")

In [ ]:
stat, p = stats.levene(vino_q_baja, vino_q_media)
print(f"Test de Levene para baja-media: Estadístico={stat:.3f}, p-valor={p:.3f}")

stat, p = stats.levene(vino_q_baja, vino_q_alta)
print(f"Test de Levene para baja-alta: Estadístico={stat:.3f}, p-valor={p:.3f}")

stat, p = stats.levene(vino_q_media, vino_q_alta)
print(f"Test de Levene para media-alta: Estadístico={stat:.3f}, p-valor={p:.3f}")




In [ ]:
# Test de Kolmogorov-Smirnov para países desarrollados
stat, p = kstest(vino_q_baja, 'norm')
print(f"Test de Kolmogorov-Smirnov para vinos baja: Estadístico={stat:.3f}, p-valor={p:.3f}")

# Test de Kolmogorov-Smirnov para países en vías de desarrollo
stat, p = kstest(vino_q_media, 'norm')
print(f"Test de Kolmogorov-Smirnov para vinos media: Estadístico={stat:.3f}, p-valor={p:.3f}")

# Test de Kolmogorov-Smirnov para países desarrollados
stat, p = kstest(vino_q_alta, 'norm')
print(f"Test de Kolmogorov-Smirnov para vinos alta: Estadístico={stat:.3f}, p-valor={p:.3f}")


In [ ]:
# Test de Mann-Whitney U para comparar Population_mln entre países desarrollados y en vías de desarrollo
stat, p = stats.mannwhitneyu(vino_q_baja, vino_q_media)
print(f"Test de Mann-Whitney U para Population_mln: Estadístico={stat:.3f}, p-valor={p:.3f}")

# Interpretación de los resultados
alpha = 0.05  # Nivel de significancia
if p > alpha:
    print("No hay suficiente evidencia para rechazar la hipótesis nula.")
    print("No hay una diferencia significativa en los vinos de calidad baja y media.")
else:
    print("Se rechaza la hipótesis nula.")
    print("Existe una diferencia significativa en los vinos de calidad baja y media.")

print("-------------------------------------------------------------------------------")

# Test de Mann-Whitney U para comparar Population_mln entre países desarrollados y en vías de desarrollo
stat, p = stats.mannwhitneyu(vino_q_baja, vino_q_alta)
print(f"Test de Mann-Whitney U para Population_mln: Estadístico={stat:.3f}, p-valor={p:.3f}")

# Interpretación de los resultados
alpha = 0.05  # Nivel de significancia
if p > alpha:
    print("No hay suficiente evidencia para rechazar la hipótesis nula.")
    print("No hay una diferencia significativa en los vinos de calidad baja y alta.")
else:
    print("Se rechaza la hipótesis nula.")
    print("Existe una diferencia significativa en los vinos de calidad baja y alta.")

print("-------------------------------------------------------------------------------")

# Test de Mann-Whitney U para comparar Population_mln entre países desarrollados y en vías de desarrollo
stat, p = stats.mannwhitneyu(vino_q_media, vino_q_alta)
print(f"Test de Mann-Whitney U para Population_mln: Estadístico={stat:.3f}, p-valor={p:.3f}")

# Interpretación de los resultados
alpha = 0.05  # Nivel de significancia
if p > alpha:
    print("No hay suficiente evidencia para rechazar la hipótesis nula.")
    print("No hay una diferencia significativa en los vinos de calidad media y alta.")
else:
    print("Se rechaza la hipótesis nula.")
    print("Existe una diferencia significativa en los vinos de calidad media y alta.")

perplexity.ai - te da fuentes?

consensus.app - evidencia a favor y en contra de una afirmacion 

elicit.ai - parecida a la anterior

In [ ]:
# Crear un boxplot para comparar la residual sugar 
sns.boxplot(x='quality', y='residual sugar', data=winesDf, notch=True)
plt.title('Comparación de residual sugar entre las distintas calidad de vinos')
plt.xlabel('calidad')
plt.ylabel('volatiles acidity')
plt.show()

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

# Escalar las características
scaler = StandardScaler()
data_scaled = scaler.fit_transform(winesDf.drop(['type', 'quality'], axis=1))

# Aplicar PCA
pca = PCA(n_components=2)
pca_components = pca.fit_transform(data_scaled)

# Gráfico de los componentes principales
plt.scatter(pca_components[:, 0], pca_components[:, 1], c=winesDf['quality'], cmap='viridis', alpha=0.7)
plt.colorbar(label='Calidad')
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.title('Reducción de Dimensionalidad con PCA')
plt.show()


In [ ]:
from sklearn.cluster import KMeans
import numpy as np

# Usar el método del codo para determinar el número óptimo de clusters
inertia = []
for k in range(1, 11):
    kmeans = KMeans(n_clusters=k, random_state=0)
    kmeans.fit(data_scaled)
    inertia.append(kmeans.inertia_)

plt.plot(range(1, 11), inertia, marker='o')
plt.xlabel('Número de Clusters')
plt.ylabel('Inercia')
plt.title('Método del Codo para el número óptimo de clusters')
plt.show()

# Supongamos que elegiste k=3 basado en el gráfico
kmeans = KMeans(n_clusters=3, random_state=0)
winesDf['cluster'] = kmeans.fit_predict(data_scaled)

# Visualización de clusters con PCA
plt.scatter(pca_components[:, 0], pca_components[:, 1], c=winesDf['cluster'], cmap='viridis', alpha=0.7)
plt.colorbar(label='Cluster')
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.title('Clustering con KMeans usando PCA')
plt.show()


In [ ]:
from sklearn.cluster import KMeans
from sklearn.manifold import TSNE
import numpy as np
import matplotlib.pyplot as plt

# Definir el número de clusters basándote en el método del codo
inertia = []
for k in range(1, 11):
    kmeans = KMeans(n_clusters=k, random_state=0)
    kmeans.fit(data_scaled)
    inertia.append(kmeans.inertia_)

# Graficar el método del codo
plt.plot(range(1, 11), inertia, marker='o')
plt.xlabel('Número de Clusters')
plt.ylabel('Inercia')
plt.title('Método del Codo para el número óptimo de clusters')
plt.show()

# Supongamos que elegiste k=4 basado en el gráfico del codo
kmeans = KMeans(n_clusters=4, random_state=0)
winesDf['cluster'] = kmeans.fit_predict(data_scaled)

# Aplicar t-SNE para reducir la dimensionalidad a 2 componentes
tsne = TSNE(n_components=2, random_state=0, perplexity=30)
tsne_components = tsne.fit_transform(data_scaled)

# Visualizar los clusters en el espacio 2D de t-SNE
plt.figure(figsize=(8, 6))
plt.scatter(tsne_components[:, 0], tsne_components[:, 1], c=winesDf['cluster'], cmap='viridis', alpha=0.7)
plt.colorbar(label='Cluster')
plt.xlabel('TSNE Component 1')
plt.ylabel('TSNE Component 2')
plt.title('Clustering con KMeans usando t-SNE')
plt.show()


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Ejemplo de gráfico de caja para comparar 'alcohol' entre clusters
plt.figure(figsize=(8, 6))
sns.boxplot(x='cluster', y='quality', data=winesDf)
plt.title('Distribución de Calidad en cada Cluster')
plt.show()


In [ ]:
# Filtra solo las columnas numéricas en el DataFrame
numeric_columns = winesDf.select_dtypes(include='number')

# Calcula el promedio de cada característica numérica para cada cluster
cluster_means = numeric_columns.groupby(winesDf['cluster']).mean()

# Genera el gráfico
import matplotlib.pyplot as plt

# Transpone los datos para que cada característica sea una serie
cluster_means.T.plot(kind='bar', figsize=(12, 8))
plt.title('Características promedio por Cluster')
plt.xlabel('Características')
plt.ylabel('Valor promedio')
plt.legend(title='Cluster')
plt.show()



In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Crea el scatter plot de 'alcohol' vs 'quality', coloreando por cluster
plt.figure(figsize=(10, 6))
sns.scatterplot(data=winesDf, x='alcohol', y='quality', hue='cluster', palette='viridis', s=60, alpha=0.7)

# Configuración de título y ejes
plt.title('Comparación de Clusters por Quality y Alcohol')
plt.xlabel('Alcohol')
plt.ylabel('Quality')
plt.legend(title='Cluster')
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Calcula la media de quality para cada cluster
cluster_quality_means = winesDf.groupby('cluster')['residual sugar'].mean()

# Genera el gráfico de barras
plt.figure(figsize=(10, 6))
cluster_quality_means.plot(kind='bar', color='skyblue', edgecolor='black')

# Configuración de título y ejes
plt.title('Residual sugar de Vino por Cluster')
plt.xlabel('Cluster')
plt.ylabel('residual sugar')
plt.xticks(rotation=0)
plt.show()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Ejemplo de gráfico de caja para comparar 'alcohol' entre clusters
plt.figure(figsize=(8, 6))
sns.boxplot(x='cluster', y='residual sugar', data=winesDf)
plt.title('Distribución de Residual Sugar en cada Cluster')
plt.show()

In [ ]:
# Ejemplo de gráfico de caja para comparar 'alcohol' entre clusters
plt.figure(figsize=(8, 6))
sns.boxplot(x='cluster', y='alcohol', data=winesDf)
plt.title('Distribución de Residual Sugar en cada Cluster')
plt.show()

In [ ]:
# Ejemplo de gráfico de caja para comparar 'alcohol' entre clusters
plt.figure(figsize=(8, 6))
sns.boxplot(x='cluster', y='density', data=winesDf)
plt.title('Distribución de Residual Sugar en cada Cluster')
plt.show()

# Ejemplo de gráfico de caja para comparar 'alcohol' entre clusters
plt.figure(figsize=(8, 6))
sns.boxplot(x='cluster', y='quality', data=winesDf)
plt.title('Distribución de Residual Sugar en cada Cluster')
plt.show()

In [ ]:
numeric_data = winesDf_copy.select_dtypes(include=[np.number])
numeric_data['cluster'] = winesDf_copy['cluster']

cluster_0_data = numeric_data[numeric_data['cluster'] == 0]
cluster_1_data = numeric_data[numeric_data['cluster'] == 1]
cluster_2_data = numeric_data[numeric_data['cluster'] == 2]
cluster_3_data = numeric_data[numeric_data['cluster'] == 3]

cluster_0_data['quality_group'] = pd.cut(cluster_0_data['quality'], 
                                         bins=[0, 5, 7, 10], 
                                         labels=['Baja', 'Media', 'Alta'])

cluster_1_data['quality_group'] = pd.cut(cluster_1_data['quality'], 
                                         bins=[0, 5, 7, 10], 
                                         labels=['Baja', 'Media', 'Alta'])

cluster_2_data['quality_group'] = pd.cut(cluster_2_data['quality'], 
                                         bins=[0, 5, 7, 10], 
                                         labels=['Baja', 'Media', 'Alta'])

cluster_3_data['quality_group'] = pd.cut(cluster_3_data['quality'], 
                                         bins=[0, 5, 7, 10], 
                                         labels=['Baja', 'Media', 'Alta'])

print(cluster_0_data.columns)  # Verifica si 'quality_group' está en las columnas


plt.figure(figsize=(8, 6))
sns.boxplot(x='quality_group', y='volatile acidity', data=cluster_0_data, notch=True)
plt.title('Distribución de volatile acidity en cada calidad para CLUSTER 0')
plt.show()

plt.figure(figsize=(8, 6))
sns.boxplot(x='quality_group', y='volatile acidity', data=cluster_1_data)
plt.title('Distribución de volatile acidity en cada calidad para CLUSTER 1')
plt.show()

plt.figure(figsize=(8, 6))
sns.boxplot(x='quality_group', y='volatile acidity', data=cluster_2_data)
plt.title('Distribución de volatile acidity en cada calidad para CLUSTER 2')
plt.show()

plt.figure(figsize=(8, 6))
sns.boxplot(x='quality_group', y='volatile acidity', data=cluster_3_data)
plt.title('Distribución de volatile acidity en cada calidad para CLUSTER 3')
plt.show()


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt


# Ejemplo de gráfico de caja para comparar 'alcohol' entre clusters
plt.figure(figsize=(8, 6))
sns.boxplot(x='cluster', y='volatile acidity', data=winesDf)
plt.title('Distribución de volatile acidity en cada Cluster')
plt.show()

In [ ]:

# Filtrar solo las columnas numéricas y añadir la columna 'cluster'
numeric_data = winesDf.select_dtypes(include=[np.number])
numeric_data['cluster'] = winesDf['cluster']

# Agrupar por cluster
numeric_data_cluster = numeric_data.groupby('cluster')

# Iterar sobre cada cluster para calcular y visualizar la matriz de correlación
for cluster, data in numeric_data_cluster:
    # Calcular la matriz de correlación
    correlation_matrix = data.drop(columns=['cluster']).corr()
    
    # Visualizar la matriz de correlación con un heatmap
    plt.figure(figsize=(10, 8))
    sns.heatmap(correlation_matrix, annot=True, cmap="coolwarm", fmt=".2f")
    plt.title(f"Matriz de correlación para el cluster {cluster}")
    plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Filtrar los datos para el cluster 3
cluster_1_data = numeric_data[numeric_data['cluster'] == 3]

# Crear el scatter plot para 'residual sugar' vs 'density'
plt.figure(figsize=(8, 6))
plt.scatter(cluster_1_data['residual sugar'], cluster_1_data['density'], alpha=0.7, color='blue')
plt.title("Scatter plot de 'residual sugar' vs 'density' para el cluster 1")
plt.xlabel("Residual Sugar")
plt.ylabel("Density")
plt.grid(True)
plt.show()

# Crear el scatter plot para 'residual sugar' vs 'density'
plt.figure(figsize=(8, 6))
plt.scatter(cluster_1_data['alcohol'], cluster_1_data['density'], alpha=0.7, color='blue')
plt.title("Scatter plot de 'density' vs 'alcohol' para el cluster 1")
plt.xlabel("alcohol")
plt.ylabel("density")
plt.grid(True)
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Filtrar los datos para el cluster 1
cluster_1_data = numeric_data[numeric_data['cluster'] == 0]

# Crear el scatter plot para 'residual sugar' vs 'density'
plt.figure(figsize=(8, 6))
plt.scatter(cluster_1_data['alcohol'], cluster_1_data['density'], alpha=0.7, color='blue')
plt.title("Scatter plot de 'alcohol' vs 'density' para el cluster 1")
plt.xlabel("alcohol")
plt.ylabel("Density")
plt.grid(True)
plt.show()


In [ ]:
from sklearn.cluster import KMeans
winesDf['cluster'] = KMeans(n_clusters=4, random_state=0).fit_predict(data_scaled)
cluster_quality = winesDf.groupby('cluster')['quality'].mean()
print(cluster_quality)

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

# Escalar las características
scaler = StandardScaler()
data_scaled = scaler.fit_transform(winesDf.drop(['type', 'quality'], axis=1))

# Aplicar PCA
pca = PCA(n_components=2)
pca_components = pca.fit_transform(data_scaled)

# Gráfico de los componentes principales
plt.scatter(pca_components[:, 0], pca_components[:, 1], c=winesDf['quality'], cmap='viridis', alpha=0.7)
plt.colorbar(label='Calidad')
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.title('Reducción de Dimensionalidad con PCA')
plt.show()

In [ ]:
winesDf_copy_acid = winesDf.copy()

# Creación de un índice de acidez
winesDf_copy_acid['acidity_index'] = (winesDf_copy_acid['volatile acidity'] + winesDf_copy_acid['citric acid'] + winesDf_copy_acid['fixed acidity'] + winesDf_copy_acid['pH'])

# Muestra los primeros valores del índice
print(winesDf_copy_acid[['volatile acidity', 'citric acid', 'fixed acidity', 'pH', 'acidity_index']].head())


VINOS CON ACIDEZ VOLATIL CERCANAS A 1g/l TIENDEN A SER DE BAJA CALIDAD



In [ ]:
# Crear un boxplot para comparar la acidez fija en ambos tipos de vino
sns.boxplot(x='quality_group', y='volatile acidity', data=winesDf_copy, notch=True)
plt.title('Comparación de alcohol entre calidad de vino')
plt.xlabel('Calidad')
plt.ylabel('alcohol')
plt.show()

vino_baja = winesDf_copy[winesDf_copy['quality_group'] == 'Baja']['volatile acidity']
vino_media = winesDf_copy[winesDf_copy['quality_group'] == 'Media']['volatile acidity']
vino_alta = winesDf_copy[winesDf_copy['quality_group'] == 'Alta']['volatile acidity']

print(vino_baja.mean())
print(vino_media.mean())
print(vino_alta.mean())



In [ ]:


# Crear un FacetGrid con 'quality_group' como columnas y mostrar la relación con 'volatile acidity' como barras
g = sns.FacetGrid(winesDf_copy, col='quality_group', height=6, aspect=1)
g.map(sns.barplot, 'quality_group', 'volatile acidity', estimator='mean', ci=None, palette='Blues')

# Añadir etiquetas y título
g.set_axis_labels('Grupo de Calidad', 'Acidez Volátil Promedio')
g.set_titles('Grupo de Calidad: {col_name}')
g.fig.suptitle('Promedio de Acidez Volátil por Grupo de Calidad', fontsize=16)
g.fig.tight_layout()
g.fig.subplots_adjust(top=0.9)  # Ajustar el título para que no se sobreponga
plt.show()

PROBAMOS NORMALIDAD POR SAPHIRO-WILK
 si el p-valor que obtenemos es menor a nuestro nivel de confianza  α  (normalmente  α=0.05 ), estamos entonces ante datos que no respetan una distribución normal:

In [ ]:
# Separamos los datos en tres grupos, de calidad baja, media y alta
vino_baja = winesDf_copy[winesDf_copy['quality_group'] == 'Baja']['volatile acidity']
vino_media = winesDf_copy[winesDf_copy['quality_group'] == 'Media']['volatile acidity']
vino_alta = winesDf_copy[winesDf_copy['quality_group'] == 'Alta']['volatile acidity']



# Test de Shapiro-Wilk para países desarrollados
stat, p = shapiro(vino_baja)
print(f"Test de Shapiro-Wilk para vinos Q baja: Estadístico={stat:.3f}, p-valor={p:.3f}")

# Test de Shapiro-Wilk para países en vías de desarrollo
stat, p = shapiro(vino_media)
print(f"Test de Shapiro-Wilk para vinos Q media: Estadístico={stat:.3f}, p-valor={p:.3f}")

# Test de Shapiro-Wilk para países en vías de desarrollo
stat, p = shapiro(vino_alta)
print(f"Test de Shapiro-Wilk para vinos Q alta: Estadístico={stat:.3f}, p-valor={p:.3f}")

In [ ]:
# QQ plot para países desarrollados
stats.probplot(vino_baja, dist="norm", plot=plt)
plt.title("QQ Plot para residual sugar en vinos garnacha")
plt.show()

# QQ plot para países en vías de desarrollo
stats.probplot(vino_media, dist="norm", plot=plt)
plt.title("QQ Plot para residual sugar en vinos riesling")
plt.show()

# QQ plot para países en vías de desarrollo
stats.probplot(vino_alta, dist="norm", plot=plt)
plt.title("QQ Plot para residual sugar en vinos riesling")
plt.show()

In [ ]:

# Test de Kolmogorov-Smirnov para países desarrollados
stat, p = kstest(vino_baja, 'norm')
print(f"Test de Kolmogorov-Smirnov para vinos baja: Estadístico={stat:.3f}, p-valor={p:.3f}")

# Test de Kolmogorov-Smirnov para países en vías de desarrollo
stat, p = kstest(vino_media, 'norm')
print(f"Test de Kolmogorov-Smirnov para vinos media: Estadístico={stat:.3f}, p-valor={p:.3f}")

# Test de Kolmogorov-Smirnov para países desarrollados
stat, p = kstest(vino_alta, 'norm')
print(f"Test de Kolmogorov-Smirnov para vinos alta: Estadístico={stat:.3f}, p-valor={p:.3f}")

In [ ]:
# Test de Mann-Whitney U para comparar Population_mln entre países desarrollados y en vías de desarrollo
stat, p = stats.mannwhitneyu(vino_baja, vino_media)
print(f"Test de Mann-Whitney U para Population_mln: Estadístico={stat:.3f}, p-valor={p:.3f}")

# Interpretación de los resultados
alpha = 0.05  # Nivel de significancia
if p > alpha:
    print("No hay suficiente evidencia para rechazar la hipótesis nula.")
    print("No hay una diferencia significativa en los vinos de calidad baja y media.")
else:
    print("Se rechaza la hipótesis nula.")
    print("Existe una diferencia significativa en los vinos de calidad baja y media.")

print("-------------------------------------------------------------------------------")

# Test de Mann-Whitney U para comparar Population_mln entre países desarrollados y en vías de desarrollo
stat, p = stats.mannwhitneyu(vino_baja, vino_alta)
print(f"Test de Mann-Whitney U para Population_mln: Estadístico={stat:.3f}, p-valor={p:.3f}")

# Interpretación de los resultados
alpha = 0.05  # Nivel de significancia
if p > alpha:
    print("No hay suficiente evidencia para rechazar la hipótesis nula.")
    print("No hay una diferencia significativa en los vinos de calidad baja y alta.")
else:
    print("Se rechaza la hipótesis nula.")
    print("Existe una diferencia significativa en los vinos de calidad baja y alta.")

print("-------------------------------------------------------------------------------")

# Test de Mann-Whitney U para comparar Population_mln entre países desarrollados y en vías de desarrollo
stat, p = stats.mannwhitneyu(vino_media, vino_alta)
print(f"Test de Mann-Whitney U para Population_mln: Estadístico={stat:.3f}, p-valor={p:.3f}")

# Interpretación de los resultados
alpha = 0.05  # Nivel de significancia
if p > alpha:
    print("No hay suficiente evidencia para rechazar la hipótesis nula.")
    print("No hay una diferencia significativa en los vinos de calidad media y alta.")
else:
    print("Se rechaza la hipótesis nula.")
    print("Existe una diferencia significativa en los vinos de calidad media y alta.")

In [ ]:
from sklearn.model_selection import train_test_split

print(winesDf_copy['quality_group'].value_counts())

# Variables predictoras y variable objetivo
X = winesDf_copy[['volatile acidity','residual sugar', 'fixed acidity', 'pH','chlorides','free sulfur dioxide','total sulfur dioxide', 'alcohol', 'sulphates','density']]  # características importantes
y = winesDf_copy['quality_group']

# Dividir el dataset en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=10)


print(f"Tamaño del conjunto de entrenamiento: {X_train.shape[0]} muestras")
print(f"Tamaño del conjunto de prueba: {X_test.shape[0]} muestras")


In [ ]:
# Ajustar StandardScaler sobre X_train
scaler = StandardScaler()
scaler.fit(X_train)

# Estandarizar X_train y X_test usando el scaler ajustado
X_train_stand = scaler.transform(X_train)
X_test_stand = scaler.transform(X_test)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Configurar el modelo de regresión logística con ajuste de pesos
model_balanced = LogisticRegression(class_weight='balanced', random_state=10, max_iter=200)
model_balanced.fit(X_train_stand, y_train)

In [ ]:
# Hacer predicciones en el conjunto de prueba
y_pred_balanced = model_balanced.predict(X_test)

In [ ]:
accuracy_balanced = accuracy_score(y_test, y_pred_balanced)
print("Precisión del modelo ajustado:", accuracy_balanced)
print("\nInforme de clasificación:\n", classification_report(y_test, y_pred_balanced))

In [ ]:
# Matriz de confusión
confusion_balanced = confusion_matrix(y_test, y_pred_balanced)
sns.heatmap(confusion_balanced, annot=True, fmt='d', cmap='Blues', xticklabels=model_balanced.classes_, yticklabels=model_balanced.classes_)
plt.xlabel('Predicho')
plt.ylabel('Verdadero')
plt.title('Matriz de Confusión (Modelo Ajustado)')
plt.show()

In [ ]:
# Obtener los coeficientes
coefficients = pd.DataFrame(model_balanced.coef_, columns=X.columns, index=['Baja', 'Media', 'Alta'])
print("Coeficientes del modelo de regresión logística:")
print(coefficients)

In [ ]:
from sklearn.linear_model import LogisticRegression

# Crear un modelo de regresión logística
logistic_model = LogisticRegression(penalty=None, random_state=10)

# Entrenar el modelo con los datos de entrenamiento estandarizados
logistic_model.fit(X_train_stand, y_train)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Asegúrate de que `theta_logistic` sea un vector unidimensional
theta_logistic = np.squeeze(theta_logistic)


# Graficar los coeficientes del modelo logístico
fig, ax = plt.subplots(figsize=(12, 6))
ax.bar(np.arange(theta_logistic.size), theta_logistic)
labels = ['volatile acidity','residual sugar', 'fixed acidity', 'pH','chlorides','free sulfur dioxide','total sulfur dioxide', 'alcohol', 'sulphates','density']  # Etiquetas para tus características

# Configurar las etiquetas de los ejes
ax.set_xticks(np.arange(theta_logistic.size))
ax.set_xticklabels(labels, rotation=45, ha='right')
ax.set_xlabel('Características')
ax.set_ylabel('Coeficiente')
ax.set_title('Coeficientes del modelo de regresión logística')
plt.tight_layout()  # Para ajustar el espacio y evitar que las etiquetas se superpongan
plt.show()

In [ ]:
print("Forma de theta_logistic:", theta_logistic.shape)


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

labels = ['volatile acidity','residual sugar', 'fixed acidity', 'pH','chlorides','free sulfur dioxide','total sulfur dioxide', 'alcohol', 'sulphates','density']  # Etiquetas para tus características


# Verifica que `theta_logistic` tenga el tamaño adecuado
if theta_logistic.shape[0] == len(labels):
    # Graficar los coeficientes del modelo logístico
    fig, ax = plt.subplots(figsize=(12, 6))
    ax.bar(np.arange(len(labels)), theta_logistic)
    
    # Configurar las etiquetas
    ax.set_xticks(np.arange(len(labels)))
    ax.set_xticklabels(labels, rotation=45, ha='right')
    ax.set_xlabel('Características')
    ax.set_ylabel('Coeficiente')
    ax.set_title('Coeficientes del modelo de regresión logística')
    plt.tight_layout()
    plt.show()
else:
    print("Error: la forma de theta_logistic no coincide con el número de características.")


In [ ]:
# Selecciona los coeficientes de la primera clase
theta_logistic_class1 = theta_logistic[0, :]

# Definir etiquetas de las características
labels = ['volatile acidity','residual sugar', 'fixed acidity', 'pH','chlorides','free sulfur dioxide','total sulfur dioxide', 'alcohol', 'sulphates','density']

# Graficar los coeficientes para la primera clase
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(12, 6))
ax.bar(np.arange(len(labels)), theta_logistic_class1)
ax.set_xticks(np.arange(len(labels)))
ax.set_xticklabels(labels, rotation=45, ha='right')
ax.set_xlabel('Características')
ax.set_ylabel('Coeficiente')
ax.set_title('Coeficientes del modelo de regresión logística (Clase 1)')
plt.tight_layout()
plt.show()


In [ ]:
# Promediar los coeficientes de todas las clases
theta_logistic_mean = theta_logistic.mean(axis=0)

# Graficar los coeficientes promedio
fig, ax = plt.subplots(figsize=(12, 6))
ax.bar(np.arange(len(labels)), theta_logistic_mean)
ax.set_xticks(np.arange(len(labels)))
ax.set_xticklabels(labels, rotation=45, ha='right')
ax.set_xlabel('Características')
ax.set_ylabel('Coeficiente promedio')
ax.set_title('Coeficientes promedio del modelo de regresión logística')
plt.tight_layout()
plt.show()


In [ ]:
from sklearn.linear_model import LogisticRegression

# Inicializar el modelo
logistic_model = LogisticRegression(max_iter=1000)

# Entrenar el modelo
logistic_model.fit(X_train_stand, y_train)

In [ ]:
from sklearn.metrics import accuracy_score, classification_report

# Predicciones en el conjunto de prueba
y_pred = logistic_model.predict(X_test)

# Calcular la precisión
accuracy = accuracy_score(y_test, y_pred)
print(f'Precisión del modelo: {accuracy:.2f}')

# Reporte de clasificación
print(classification_report(y_test, y_pred, target_names=['Baja', 'Media', 'Alta']))

In [ ]:
# Obtener los coeficientes
coefficients = pd.DataFrame(model_balanced.coef_, columns=X.columns, index=['Baja', 'Media', 'Alta'])
print("Coeficientes del modelo de regresión logística:")
print(coefficients)


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

# Crear un modelo de Regresión Logística baseline
baseline_logistic_model = LogisticRegression(penalty=None, random_state=10)

# Realizar la validación cruzada y calcular la precisión
cv_scores = cross_val_score(baseline_logistic_model, X_train_stand, y_train, cv=5, scoring='accuracy')

# Imprimir los resultados de la validación cruzada
print("Exactitud en cada fold:", cv_scores)
print("Exactitud promedio:", cv_scores.mean())
print("Desvío de la Exactitud:", cv_scores.std())

# Dejamos el modelo entrenado al final
baseline_logistic_model.fit(X_train_stand, y_train)

In [ ]:
from sklearn.model_selection import GridSearchCV

# Definir los valores de alpha que queremos probar
param_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100]}

# Crear un modelo de Regresión Logística
logistic_model = LogisticRegression(penalty='l2', max_iter=1000, random_state=10)

# Crear un objeto GridSearchCV
grid_search = GridSearchCV(logistic_model, param_grid, cv=5, scoring='accuracy')

# Ajustar el modelo a los datos de entrenamiento
grid_search.fit(X_train_stand, y_train)

# Imprimir los mejores parámetros encontrados
print("Mejores parámetros:", grid_search.best_params_)
# Imprimir la mejor exactitud encontrada
print("Mejor exactitud:", grid_search.best_score_)

In [ ]:
# Obtener el mejor modelo
best_logistic_model = grid_search.best_estimator_

In [ ]:
import seaborn as sns
from sklearn.metrics import accuracy_score, confusion_matrix

# Predecir con el modelo original en X_test_stand
y_pred_original = baseline_logistic_model.predict(X_test_stand)


# Define tus etiquetas de calidad
labels = ["Baja", "Media", "Alta"]

# Calcular la exactitud y la matriz de confusión del modelo original
accuracy_original = accuracy_score(y_test, y_pred_original)
confusion_matrix_original = confusion_matrix(y_test, y_pred_original)
sns.heatmap(confusion_matrix_original, annot=True, fmt='d', cmap='Blues', xticklabels= labels, yticklabels= labels)
plt.xlabel('Predicho')
plt.ylabel('GT')
plt.title('Matriz confusión (modelo sin regularizar)')
plt.show()

# Predecir con el mejor modelo regularizado en X_test_stand
y_pred_best = best_logistic_model.predict(X_test_stand)

# Calcular la exactitud y la matriz de confusión del mejor modelo regularizado
accuracy_best = accuracy_score(y_test, y_pred_best)
confusion_matrix_best = confusion_matrix(y_test, y_pred_best)
sns.heatmap(confusion_matrix_best, annot=True, fmt='d', cmap='Blues', xticklabels= labels, yticklabels= labels)
plt.xlabel('Predicho')
plt.ylabel('GT')
plt.title('Matriz confusión (modelo regularizado)')
plt.show()

# Imprimir los resultados
print("Exactitud del modelo original:", accuracy_original)
print("Exactitud del mejor modelo regularizado:", accuracy_best)

In [ ]:
import pandas as pd

# Supongamos que tu DataFrame original se llama winesDf
# Crear una copia del DataFrame
winesDf_copy_aux = winesDf.copy()

# Añadir la columna 'quality_group' en la copia
winesDf_copy_aux['quality_group'] = pd.cut(winesDf_copy_aux['quality'], 
                                       bins=[0, 5, 10], 
                                       labels=['Baja', 'Alta'])

In [ ]:
print(winesDf_copy_aux['quality_group'].value_counts())

In [ ]:
from sklearn.cluster import KMeans
from sklearn.manifold import TSNE
import numpy as np
import matplotlib.pyplot as plt

# Definir el número de clusters basándote en el método del codo
inertia = []
for k in range(1, 11):
    kmeans = KMeans(n_clusters=k, random_state=0)
    kmeans.fit(data_scaled)
    inertia.append(kmeans.inertia_)

# Graficar el método del codo
plt.plot(range(1, 11), inertia, marker='o')
plt.xlabel('Número de Clusters')
plt.ylabel('Inercia')
plt.title('Método del Codo para el número óptimo de clusters')
plt.show()

# Supongamos que elegiste k=4 basado en el gráfico del codo
kmeans = KMeans(n_clusters=4, random_state=0)
winesDf_copy['clusterA'] = kmeans.fit_predict(data_scaled)

# Aplicar t-SNE para reducir la dimensionalidad a 2 componentes
tsne = TSNE(n_components=2, random_state=0, perplexity=30)
tsne_components = tsne.fit_transform(data_scaled)

# Visualizar los clusters en el espacio 2D de t-SNE
plt.figure(figsize=(8, 6))
plt.scatter(tsne_components[:, 0], tsne_components[:, 1], c=winesDf_copy['clusterA'], cmap='viridis', alpha=0.7)
plt.colorbar(label='ClusterA')
plt.xlabel('TSNE Component 1')
plt.ylabel('TSNE Component 2')
plt.title('Clustering con KMeans usando t-SNE')
plt.show()

In [ ]:
print((winesDf['volatile acidity'] >= 1).value_counts())

In [ ]:
# Filtra los vinos con volatile acidity mayores a 1
vinos_volatiles_altos = winesDf_copy[winesDf_copy['volatile acidity'] >= 1]

# Contar cuántos vinos de alta acidez volátil están en cada cluster
distribucion_clusters = vinos_volatiles_altos['clusterA'].value_counts()
print(distribucion_clusters)

In [ ]:
numeric_data0 = winesDf_copy.select_dtypes(include=[np.number])
numeric_data0['clusterA'] = winesDf_copy['clusterA']

# Agrupar por cluster
numeric_data_cluster = numeric_data0.groupby('clusterA')

In [ ]:
cluster_0_data = winesDf_copy[winesDf_copy['clusterA'] == 0]


vinos_volatiles_altos = cluster_0_data[cluster_0_data['volatile acidity'] >= 1]

# Ver la distribución de la calidad entre los vinos con alta acidez volátil
distribucion_calidad = vinos_volatiles_altos['quality_group'].value_counts()
print(distribucion_calidad)


# Agrupar por quality_group y calcular los promedios de otras características
#promedios_por_calidad = vinos_volatiles_altos.groupby('quality_group').mean()
#print(promedios_por_calidad)


# de 0 a 4 CALIDAD BAJA
# de 5 a 6 CALIDAD MEDIA
# de 7 a 10 CALIDAD ALTA


A MAYOR RESIDUAL SUGAR HAY MAYOR CONTENIDO DE ALCOHOL Y AUMENTA LA DENSIDAD.

VER VALORES EXTREMOS DE DENSIDAD



In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

# Selección de las variables
X = winesDf_copy[['residual sugar', 'density', 'alcohol']]  # Variables independientes
y = winesDf_copy['quality']  # Variable dependiente

# Discretizamos la variable objetivo para la estratificación
y_binned = pd.cut(y, bins=100, labels=False)

# Dividir los datos en entrenamiento y prueba (70% - 30%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=10, stratify = y_binned)

# Repetimos ell estratificado con y_train porque no tenemos los índices del estratificado anterior
y_train_binned = pd.cut(y_train, bins=100, labels=False)

# y aplicamos nuevamente la función pero sobre los datos de entrenamiento, esta vez
# para extraer los datos de validación, tomando el 10% para ese conjunto
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train,
                                                  test_size=0.1,
                                                  random_state=10, # usamos una semilla aleatoria para ser reproducibles
                                                  stratify = y_train_binned)

#plt.hist(y_train, bins=20, label='y_train')
#plt.hist(y_test, bins=20, label='y_test')
#plt.hist(y_val, bins=20, label='y_val')
#plt.legend(loc='upper right')
#plt.title('Histograma de la variable objetivo en los conjuntos de datos')
#plt.show()

# creamos un standard scaler
scaler = StandardScaler()
# y lo fiteamos con los datos de entrenamiento
scaler.fit(X_train)

# estandarizamos las muestras de entrenamiento
X_train_stand = scaler.transform(X_train)

# Creación del modelo de regresión lineal
lin_model = LinearRegression(fit_intercept=True)
lin_model.fit(X_train_stand, y_train)

# agregamos el scaler como un atributo al objeto para no perderlo
lin_model.standardizer_ = scaler

# estandarizamos los datos de validación y predecimos
X_val_stand = lin_model.standardizer_.transform(X_val)
y_pred_val = lin_model.predict(X_val_stand)
# mostramos lo que obtuvimos
#print("Valores obtenidos: {}".format(y_pred_val))
# y lo que esperábamos obtener
#print("Valores esperados: {}".format(np.array(y_val)))

import math
from sklearn.metrics import mean_squared_error, mean_absolute_error

# calculamos las métricas
mse = mean_squared_error(y_val, y_pred_val)
rmse = math.sqrt(mse)
mae = mean_absolute_error(y_val, y_pred_val)

# las imprimimos por pantalla
print("MSE = {:.4f}".format(mse))
print("RMSE = {:.4f}".format(rmse))
print("MAE = {:.4f}".format(mae))
print('--------------------------------------------------------------------------')


# Predecir en X_train_stand y X_val_stand
y_pred_train = lin_model.predict(X_train_stand)
y_pred_val = lin_model.predict(X_val_stand)

# Calcular el MAE para entrenamiento y validación
mae_train = mean_absolute_error(y_train, y_pred_train)
mae_val = mean_absolute_error(y_val, y_pred_val)

print("MAE en entrenamiento: {:.4f}".format(mae_train))
print("MAE en validación: {:.4f}".format(mae_val))
print('--------------------------------------------------------------------------')


from sklearn.linear_model import Ridge

# Creamos un modelo de Ridge Regression con un valor alto de alpha
ridge_model_high_alpha = Ridge(alpha=1000)
# Le asociamos el scaler así no lo perdemos
ridge_model_high_alpha.standardizer_ = scaler
# Entrenamos el modelo
ridge_model_high_alpha.fit(X_train_stand, y_train)

# Predecir en X_val_stand con el modelo Ridge
y_pred_val_ridge_high_alpha = ridge_model_high_alpha.predict(X_val_stand)

# Calcular el MAE para el modelo Ridge
mae_val_ridge_high_alpha = mean_absolute_error(y_val, y_pred_val_ridge_high_alpha)

print("MAE en validación (modelo convencional): {:.4f}".format(mae_val))
print("MAE en validación (Ridge, alpha alto): {:.4f}".format(mae_val_ridge_high_alpha))
print('--------------------------------------------------------------------------')

# Definimos los valores de alpha que queremos probar
alphas = [0.001, 0.01, 0.1, 1, 10, 100, 1000]

# Creamos una lista vacía para almacenar los errores MAE para cada valor de alpha
mae_list = []

# Iteramos sobre los valores de alpha
for alpha in alphas:
  # Creamos un modelo de Ridge Regression con el valor de alpha actual
  ridge_model = Ridge(alpha=alpha)
  ridge_model.standardizer_ = scaler
  # Entrenamos el modelo
  ridge_model.fit(X_train_stand, y_train)
  # Predecimos en X_val_stand
  y_pred_val_ridge = ridge_model.predict(X_val_stand)
  # Calculamos el MAE
  mae_val_ridge = mean_absolute_error(y_val, y_pred_val_ridge)
  # Almacenamos el error MAE
  mae_list.append(mae_val_ridge)


# Encontrar el mejor alpha
best_alpha = alphas[np.argmin(mae_list)]
print("El mejor alpha es: ", best_alpha)

# Entrenar el modelo final con el mejor alpha
ridge_model_best_alpha = Ridge(alpha=best_alpha)
ridge_model_best_alpha.standardizer_ = scaler
ridge_model_best_alpha.fit(X_train_stand, y_train)
# Predecir en X_val_stand con el mejor modelo
y_pred_val_ridge_best_alpha = ridge_model_best_alpha.predict(X_val_stand)
# Calcular el MAE para el mejor modelo
mae_val_ridge_best_alpha = mean_absolute_error(y_val, y_pred_val_ridge_best_alpha)

print("MAE en validación (modelo convencional): {:.5f}".format(mae_val))
print("MAE en validación (Ridge, alpha alto): {:.5f}".format(mae_val_ridge_high_alpha))
print("MAE en validación (Ridge, mejor alpha): {:.5f}".format(mae_val_ridge_best_alpha))
print('--------------------------------------------------------------------------')

from sklearn.linear_model import Lasso

# Creamos un modelo de LASSO con un valor alto de alpha
lasso_model_high_alpha = Lasso(alpha=1000)
# Le asociamos el scaler así no lo perdemos
lasso_model_high_alpha.standardizer_ = scaler
# Entrenamos el modelo
lasso_model_high_alpha.fit(X_train_stand, y_train)

# Recuperamos los coeficientes del modelo LASSO
theta_lasso_high_alpha = lasso_model_high_alpha.coef_
# Y nos traemos ya que estamos los del modelo con el mejor alpha
theta_ridge_best_alpha = ridge_model_best_alpha.coef_


# Definimos los valores de alpha que queremos probar
alphas = [0.001, 0.01, 0.1, 1, 10, 100]

# Creamos una lista vacía para almacenar los errores MAE para cada valor de alpha
mae_list_lasso = []

# Iteramos sobre los valores de alpha
for alpha in alphas:
  # Creamos un modelo de Lasso Regression con el valor de alpha actual
  lasso_model = Lasso(alpha=alpha)
  lasso_model.standardizer_ = scaler
  # Entrenamos el modelo
  lasso_model.fit(X_train_stand, y_train)
  # Predecimos en X_val_stand
  y_pred_val_lasso = lasso_model.predict(X_val_stand)
  # Calculamos el MAE
  mae_val_lasso = mean_absolute_error(y_val, y_pred_val_lasso)
  # Almacenamos el error MAE
  mae_list_lasso.append(mae_val_lasso)

# Encontrar el mejor alpha
best_alpha_lasso = alphas[np.argmin(mae_list_lasso)]
print("El mejor alpha para Lasso es: ", best_alpha_lasso)

# Entrenar el modelo final con el mejor alpha
lasso_model_best_alpha = Lasso(alpha=best_alpha_lasso)
lasso_model_best_alpha.standardizer_ = scaler
lasso_model_best_alpha.fit(X_train_stand, y_train)

# Predecir en X_val_stand con el mejor modelo
y_pred_val_lasso_best_alpha = lasso_model_best_alpha.predict(X_val_stand)

# Calcular el MAE para el mejor modelo
mae_val_lasso_best_alpha = mean_absolute_error(y_val, y_pred_val_lasso_best_alpha)

print("MAE en validación (modelo convencional): {:.5f}".format(mae_val))
print("MAE en validación (Ridge, mejor alpha): {:.5f}".format(mae_val_ridge_best_alpha))
print("MAE en validación (Lasso, mejor alpha): {:.5f}".format(mae_val_lasso_best_alpha))
print('--------------------------------------------------------------------------')



# Estandarizar X_test
X_test_stand = scaler.transform(X_test)

# Evaluar el modelo convencional en X_test_stand
y_pred_test = lin_model.predict(X_test_stand)
mae_test = mean_absolute_error(y_test, y_pred_test)
mse_test = mean_squared_error(y_test, y_pred_test)
rmse_test = math.sqrt(mse_test)

# Evaluar Ridge con mejor alpha en X_test_stand
y_pred_test_ridge = ridge_model_best_alpha.predict(X_test_stand)
mae_test_ridge = mean_absolute_error(y_test, y_pred_test_ridge)
mse_test_ridge = mean_squared_error(y_test, y_pred_test_ridge)
rmse_test_ridge = math.sqrt(mse_test_ridge)



# Predicción
#y_pred = lin_model.predict(X_test)

# Evaluación del modelo
#mse = mean_squared_error(y_test, y_pred)
#r2 = r2_score(y_test, y_pred)

#print(f"Mean Squared Error: {mse}")
#print(f"R2 Score: {r2}")

# Visualización de los coeficientes del modelo
#coef_df = pd.DataFrame(lin_model.coef_, X.columns, columns=['Coeficiente'])
#print(coef_df)

# Graficar la relación entre las predicciones y los valores reales
#plt.scatter(y_test, y_pred)
#plt.xlabel("Valores reales")
#plt.ylabel("Predicciones")
#plt.title("Predicciones vs Reales (Regresión Lineal)")
#plt.show()


In [ ]:
# Evaluar Lasso con mejor alpha en X_test_stand
y_pred_test_lasso = lasso_model_best_alpha.predict(X_test_stand)
mae_test_lasso = mean_absolute_error(y_test, y_pred_test_lasso)
mse_test_lasso = mean_squared_error(y_test, y_pred_test_lasso)
rmse_test_lasso = math.sqrt(mse_test_lasso)

print("MAE en test:")
print("Modelo convencional: {:.5f}".format(mae_test))
print("Ridge (mejor alpha): {:.5f}".format(mae_test_ridge))
print("Lasso (mejor alpha): {:.5f}".format(mae_test_lasso))

print("\nMSE en test:")
print("Modelo convencional: {:.5f}".format(mse_test))
print("Ridge (mejor alpha): {:.5f}".format(mse_test_ridge))
print("Lasso (mejor alpha): {:.5f}".format(mse_test_lasso))

print("\nRMSE en test:")
print("Modelo convencional: {:.5f}".format(rmse_test))
print("Ridge (mejor alpha): {:.5f}".format(rmse_test_ridge))
print("Lasso (mejor alpha): {:.5f}".format(rmse_test_lasso))

In [ ]:
# Predicción
y_pred = lin_model.predict(X_test)

# Evaluación del modelo
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"R2 Score: {r2}")

# Visualización de los coeficientes del modelo
coef_df = pd.DataFrame(lin_model.coef_, X.columns, columns=['Coeficiente'])
print(coef_df)

# Graficar la relación entre las predicciones y los valores reales
plt.scatter(y_test, y_pred)
plt.xlabel("Valores reales")
plt.ylabel("Predicciones")
plt.title("Predicciones vs Reales (Regresión Lineal)")
plt.show()


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# Selección de las variables para el modelo logístico
X_log = winesDf_copy[['residual sugar', 'density', 'alcohol']]  # Variables independientes
y_log = winesDf_copy['quality_group']  # Variable dependiente

# Dividir los datos en entrenamiento y prueba (70% - 30%)
X_train_log, X_test_log, y_train_log, y_test_log = train_test_split(X_log, y_log, test_size=0.3, random_state=10)

# estandarizamos las muestras de entrenamiento
#X_train_stand_log = scaler.transform(X_train_log)

# Creación del modelo de regresión logística
log_model = LogisticRegression(max_iter=1000)
log_model.fit(X_train_log, y_train_log)

# Predicción
y_pred_log = log_model.predict(X_test_log)

# Evaluación del modelo
accuracy = accuracy_score(y_test_log, y_pred_log)
print(f"Precisión del modelo logístico: {accuracy}")
print("\nReporte de clasificación:")
print(classification_report(y_test_log, y_pred_log))

# Visualización de los coeficientes del modelo
coef_df_log = pd.DataFrame(log_model.coef_.T, X_log.columns, columns=['Coeficiente'])
print(coef_df_log)

# Graficar la relación entre las predicciones y los valores reales
plt.scatter(y_test_log, y_pred_log)
plt.xlabel("Valores reales")
plt.ylabel("Predicciones")
plt.title("Predicciones vs Reales (Regresión Logística)")
plt.show()

In [ ]:
from sklearn.cluster import KMeans

# Asumiendo que ya tienes el modelo de clustering KMeans ajustado
# Supongamos que ya tienes las predicciones de clústeres en la columna 'cluster'

# Seleccionamos los vinos con las características de interés
X_cluster = winesDf_copy[['residual sugar', 'density', 'alcohol']]

# Ajustar el modelo KMeans (si no lo tienes ajustado ya)
kmeans = KMeans(n_clusters=3, random_state=10)
winesDf_copy['cluster'] = kmeans.fit_predict(X_cluster)

# Ver cómo se distribuyen los vinos en cada clúster según las características
sns.pairplot(winesDf_copy, vars=['residual sugar', 'density', 'alcohol'], hue='cluster')
plt.show()

# Analizar la distribución de calidad en cada clúster
for cluster_num in winesDf_copy['cluster'].unique():
    print(f"Distribución de calidad en el clúster {cluster_num}:")
    print(winesDf_copy[winesDf_copy['cluster'] == cluster_num]['quality_group'].value_counts())
    print("\n")